In [15]:
!pip install -q confluent-kafka[avro] avro-python3

In [25]:
from confluent_kafka import DeserializingConsumer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroDeserializer
from confluent_kafka.serialization import StringDeserializer

# Kafka configuration
kafka_conf = {
    'bootstrap.servers': 'cluster-kafka-brokers.kafka:9092',
    'group.id': 'notebook-next',
    'auto.offset.reset': 'earliest',
}

# Schema Registry configuration
schema_registry_conf = {
    'url': 'http://schema-registry.kafka:8081'
}

schema_registry_client = SchemaRegistryClient(schema_registry_conf)

# Deserializers
value_deserializer = AvroDeserializer(schema_registry_client)
key_deserializer = AvroDeserializer(schema_registry_client)

consumer_conf = {**kafka_conf,
                 'key.deserializer': key_deserializer,
                 'value.deserializer': value_deserializer}

consumer = DeserializingConsumer(consumer_conf)

# consumer.subscribe(['orders.public.orders'])

consumer.subscribe(['order_reviews.ecommerce.order_reviews'])

consumer

In [26]:
messages = []
for i in range(5):
    msg = consumer.poll()
    # messages.append(msg.value())
    messages.append({"key": msg.key(), "value": msg.value()})

messages

# TODO: opensearch kafka sink connector can't parse json with non UTF-32 characters

[{'key': '{"_id": "66a7704facbba45dda61e406"}',
  'value': '{"_id": "66a7704facbba45dda61e406", "order_id": "9021fdebb45ac827a6033d6969d5c2e3", "review_id": "ef468ee6d6910150dc4ea20c19ad116d", "review_answer_timestamp": "2017-01-16T00:02:47Z", "review_comment_message": "Pensava que era maior, mas tbm não tinha noção de 20 cm. Fora isso lindinha amei , tudo perfeitinho , estava pra chegar dia 09/02 mas veio dia 13/01 que bom pq eu queria pra 27/01 níver de minha filha", "review_comment_title": null, "review_creation_date": "2017-01-14T00:00:00Z", "review_score": 4}'},
 {'key': '{"_id": "66a7704facbba45dda61e40d"}',
  'value': '{"_id": "66a7704facbba45dda61e40d", "order_id": "157ec3dc3f38cdbd2706bd216edfe8fb", "review_id": "6bd763b0a18bc7c57cf36289e4946548", "review_answer_timestamp": "2017-01-14T19:55:44Z", "review_comment_message": "Comprei 2 produtos no mesmo pedido com os srs 1 chegou falta mais 1 .Fiquei surpreso em ver à nota fiscal com um único produto já que jogar pedido tem 2 ."

In [27]:
messages = []
for i in range(5):
    msg = consumer.poll()
    messages.append({"key": msg.key(), "value": msg.value()})

messages

[{'key': {'order_id': '2f9d791088532b218b41f892cc16073b'},
  'value': {'order_id': '2f9d791088532b218b41f892cc16073b',
   'customer_id': '937bb74713a47912471275b234236764',
   'order_status': 'delivered',
   'order_purchase_timestamp': 1483624128000000,
   'order_approved_at': 1483760121000000,
   'order_delivered_carrier_date': 1484150933000000,
   'order_delivered_customer_date': 1484672266000000,
   'order_estimated_delivery_date': 1486944000000000}},
 {'key': {'order_id': '7004296aa0256632eaddc171edaf727f'},
  'value': {'order_id': '7004296aa0256632eaddc171edaf727f',
   'customer_id': '3a7856f2a103364d279de7f72e18b7c4',
   'order_status': 'delivered',
   'order_purchase_timestamp': 1483623082000000,
   'order_approved_at': 1483760669000000,
   'order_delivered_carrier_date': 1484148953000000,
   'order_delivered_customer_date': 1484582721000000,
   'order_estimated_delivery_date': 1485907200000000}},
 {'key': {'order_id': '0bda8164c1a12b6a388ebec8559ee287'},
  'value': {'order_id':

In [1]:
import polars as pl
import sqlalchemy as sa
from trino.dbapi import connect

In [2]:
stream = connect(host="trino.sql", port=80, user="admin", catalog="stream", schema="default")
def execute(query: str): return pl.read_database(query, connection=stream)

In [3]:
execute("show schemas")

Schema
str
"""default"""
"""information_schema"""


In [4]:
execute("show tables from default")

Table
str
"""order_items.public.order_items"""
"""order_payments.public.order_pa…"
"""order_reviews.ecommerce.order_…"
"""orders.public.orders"""


In [21]:
# TODO:
# TrinoQueryError: TrinoQueryError(type=INTERNAL_ERROR, name=GENERIC_INTERNAL_ERROR, message="Multiple entries with same key:
#     order_id=KafkaColumnHandle{name=order_id, type=varchar, mapping=order_id, dataFormat=null, formatHint=null, keyCodec=false, hidden=false, internal=false}
# and order_id=KafkaColumnHandle{name=order_id, type=varchar, mapping=order_id, dataFormat=null, formatHint=null, keyCodec=true, hidden=false, internal=false}",
# query_id=20240727_111738_00013_rkr7m)
execute('select order_purchase_timestamp from "orders.public.orders"')

In [10]:
opensearch = connect(host="trino.sql", port=80, user="admin", catalog="opensearch", schema="default")
def exc(query: str): return pl.read_database(query, connection=opensearch)

In [30]:
exc("show schemas")

Schema
str
"""default"""
"""information_schema"""
"""system"""


In [31]:
exc("show tables from default")

Table
str
""".kibana"""
""".kibana_1"""
""".opensearch-observability"""
""".plugins-ml-config"""
""".ql-datasources"""
"""orders.public.orders"""


In [23]:
df = exc('select * from "orders.public.orders"')
df.head(5)

customer_id,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_id,order_purchase_timestamp,order_status
str,i64,i64,i64,i64,str,i64,str
"""24cf4347382b8e190a512b2de912f8…",1485364523000000,1485526881000000,1485865358000000,1488412800000000,"""20de77e28f73070461781b0eb6228f…",1485364000000000,"""delivered"""
"""0792343f0465acf2c7aee7e4fce852…",1485797482000000,1485957468000000,1487248523000000,1488844800000000,"""c6522dab7d6b83ada4140b653c4db0…",1485792458000000,"""delivered"""
"""c3ab6358d2556844d38ed5dbab0b41…",1485869605000000,1486028254000000,null,1490054400000000,"""b5b103fe531168050d785d5b8de741…",1485782039000000,"""shipped"""
"""bf6418c67d6b29aada7d4a9a0cfb3e…",1485837082000000,null,null,1489104000000000,"""2e287cee637969033ec9ab49602e61…",1485791333000000,"""approved"""
"""91a10aff1998894aefda53b691a5d6…",1485837524000000,1485855501000000,1486965536000000,1489622400000000,"""24040647efd179bd506acacbbbabd8…",1485719960000000,"""delivered"""


In [17]:
exc('select count(*) from "orders.public.orders"')

_col0
i64
2134


In [18]:
exc('select count(*) from "orders.public.orders"')

_col0
i64
2249


In [19]:
exc('select count(*) from "orders.public.orders"')

_col0
i64
2256


In [22]:
exc('select count(*) from "orders.public.orders"')

_col0
i64
2491


In [28]:
exc('select count(*) from "orders.public.orders"')

_col0
i64
4802


In [ ]:
exc('select count(*) from "orders.public.order_reviews"')